# Etapa 1

Montar o Google Drive

In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Etapa 2

Definir os caminhos dos arquivos

In [72]:
import os
import glob

pasta_input = '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input'

# Busca o arquivo XLSX
arquivo_xlsx = None  # Inicializa arquivo_xlsx
arquivos_xlsx = glob.glob(os.path.join(pasta_input, '*.xlsx'))
if arquivos_xlsx:
    arquivo_xlsx = arquivos_xlsx[0]  # Pega o primeiro arquivo XLSX encontrado
    print(f'Arquivo XLSX encontrado: {arquivo_xlsx}')
else:
    print('Nenhum arquivo XLSX encontrado na pasta.')

# Busca todos os arquivos CSV
arquivos_csv = glob.glob(os.path.join(pasta_input, '*.csv'))
if arquivos_csv:
    print(f'Arquivos CSV encontrados:')
    for arquivo_csv in arquivos_csv:
        print(f'  - {arquivo_csv}')
else:
    print('Nenhum arquivo CSV encontrado na pasta.')

Arquivo XLSX encontrado: /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input/BB_extrato.xlsx
Arquivos CSV encontrados:
  - /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input/2025-05-extrato.csv


# Etapa 3

Ler o arquivo XLSX

In [73]:
import pandas as pd

try:
    df_xlsx = pd.read_excel(arquivo_xlsx)
    print('\nArquivo XLSX lido com sucesso:')
    #print(df_xlsx.head())  # Exibe as primeiras linhas do DataFrame do XLSX

    # Obtém a lista de colunas do DataFrame do XLSX
    colunas_xlsx = df_xlsx.columns.tolist()
    # print(f'\nColunas do arquivo XLSX: {colunas_xlsx}')

except FileNotFoundError:
    print(f'Erro: Arquivo XLSX não encontrado em: {arquivo_xlsx}')
    df_xlsx = pd.DataFrame() # Cria um DataFrame vazio para evitar erros posteriores
    colunas_xlsx = []
except Exception as e:
    print(f'Ocorreu um erro ao ler o arquivo XLSX: {e}')
    df_xlsx = pd.DataFrame() # Cria um DataFrame vazio para evitar erros posteriores
    colunas_xlsx = []


Arquivo XLSX lido com sucesso:
    MesAno        Data Dependencia Origem            Histórico  \
0  2019-02  2019-02-25                NaN       Saldo Anterior   
1  2022-03  2022-03-30                NaN        Pix - Enviado   
2  2022-03  2022-03-30                NaN       Saldo Anterior   
3  2022-03  2022-03-31                NaN            S A L D O   
4  2022-04  2022-04-07                NaN  BB RF LP Automático   

     Data do Balancete  Número do documento   Valor idCategoria  \
0                  NaN                    0  140.44         NaN   
1  2022-03-30 00:00:00                33001 -120.00      Outr03   
2                  NaN                    0   20.44         NaN   
3                  NaN                    0   20.44         NaN   
4  2022-04-07 00:00:00              1200002  187.20       Rec09   

          Observação  idFormaPagto  idFluxo  \
0                NaN           NaN      NaN   
1   não identificado           NaN      NaN   
2                NaN       

# Etapa 4

Ler os arquivos CSV, realizar transformações e formatar para as colunas do XLSX

In [74]:
import pandas as pd

arquivos_csv = glob.glob(os.path.join(pasta_input, '*.csv'))

if arquivos_csv:
    df_csv_list = []
    for arquivo_csv in arquivos_csv:
        try:
            df_temp = pd.read_csv(arquivo_csv, sep=',', encoding='latin-1')
            print(f'Arquivo CSV lido com sucesso (com encoding latin-1): {arquivo_csv}')

            # Excluir "Unnamed: 6"
            if 'Unnamed: 6' in df_temp.columns:
                df_temp = df_temp.drop(columns=['Unnamed: 6'])
                print(f"Coluna 'Unnamed: 6' excluída do arquivo: {arquivo_csv}")

            # Formatar 'Data' e criar 'anomes'
            if 'Data' in df_temp.columns:
                try:
                    # Força a coluna 'Data' a ser string
                    df_temp['Data'] = df_temp['Data'].astype(str)
                    print(f"Coluna 'Data' convertida para string no arquivo: {arquivo_csv}")
                    # Criação da coluna 'anomes' pegando os 7 primeiros caracteres (aaaa-mm)
                    df_temp['MesAno'] = pd.to_datetime(df_temp['Data'], format='%d/%m/%Y', errors='coerce').dt.strftime('%Y-%m')
                    print(f"Coluna 'anomes' criada no formato 'aaaa-mm' no arquivo: {arquivo_csv}")
                    # Tentativa de converter e reformatar 'Data' para 'aaaa-mm-dd' para o 'Indice' (após criar 'anomes')
                    df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce').dt.strftime('%Y-%m-%d')
                except Exception as e:
                    print(f"Aviso: Erro ao processar 'Data' ou criar 'anomes' no arquivo {arquivo_csv}: {e}. Verifique a coluna 'Data'.")
                    df_temp['anomes'] = None
            else:
                print(f"Aviso: Coluna 'Data' não encontrada em: {arquivo_csv}")
                df_temp['anomes'] = None

            # Criar 'Indice'
            if all(col in df_temp.columns for col in ['Data', 'Histórico', 'Número do documento', 'Valor']):
                df_temp['Indice'] = df_temp['Data'].astype(str) + '-' + \
                                     df_temp['Histórico'].astype(str) + '-' + \
                                     df_temp['Número do documento'].astype(str) + '-' + \
                                     df_temp['Valor'].astype(str)
                print(f"Coluna 'Indice' criada no arquivo: {arquivo_csv}")
            else:
                print(f"Aviso: Colunas para 'Indice' não encontradas em: {arquivo_csv}")

            # Incluir colunas extras (inicialmente com None)
            df_temp['idCategoria'] = None
            df_temp['Observação'] = None
            df_temp['idFormaPagto'] = None
            df_temp['idFluxo'] = None
            print(f"Colunas extras adicionadas ao arquivo: {arquivo_csv}")

            # Formatar df_temp para ter as mesmas colunas e na mesma ordem do df_xlsx
            if 'df_xlsx' in locals() and not df_xlsx.empty: # Verifica se df_xlsx foi lido corretamente
                colunas_xlsx = df_xlsx.columns.tolist()
                df_temp = df_temp.reindex(columns=colunas_xlsx)
                print(f"\nDataFrame do arquivo CSV formatado para as colunas do XLSX: {arquivo_csv}")
                # print(df_temp.head())
            else:
                print("\nAviso: As colunas do XLSX não foram carregadas corretamente. O DataFrame do CSV não foi formatado para corresponder.")

            df_csv_list.append(df_temp)

        except FileNotFoundError:
            print(f'Erro: Arquivo CSV não encontrado: {arquivo_csv}')
        except Exception as e:
            print(f'Ocorreu um erro ao ler o arquivo CSV: {arquivo_csv}. Erro: {e}')

    if df_csv_list:
        df_csv = pd.concat(df_csv_list, ignore_index=True)
        print("\nTodos os arquivos CSV foram lidos e concatenados com sucesso, formatados para as colunas do XLSX.")
        # print(df_csv.head())
    else:
        print('\nNenhum arquivo CSV foi lido com sucesso.')
        df_csv = pd.DataFrame()
else:
    print('Nenhum arquivo CSV encontrado na pasta.')
    df_csv = pd.DataFrame()

Arquivo CSV lido com sucesso (com encoding latin-1): /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input/2025-05-extrato.csv
Coluna 'Unnamed: 6' excluída do arquivo: /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input/2025-05-extrato.csv
Coluna 'Data' convertida para string no arquivo: /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input/2025-05-extrato.csv
Coluna 'anomes' criada no formato 'aaaa-mm' no arquivo: /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input/2025-05-extrato.csv
Coluna 'Indice' criada no arquivo: /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input/2025-05-extrato.csv
Colunas extras adicionadas ao arquivo: /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input/2025-05-extrato.csv

DataFrame do arquivo CSV formatado para as colunas do XLSX: /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input/2025-05-extrato.csv
    MesAno        Data Dependencia Origem  \
0  2025-04  2025-04-29           

<ipython-input-74-18f409ff5c30>:27: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce').dt.strftime('%Y-%m-%d')


# Etapa 5

Identificar e adicionar novos registros ao df_xlsx

In [75]:
# Garante que a coluna 'Indice' exista em ambos os DataFrames
if 'Indice' in df_xlsx.columns and 'Indice' in df_csv.columns:
    # Identifica os 'Indice' presentes em df_csv mas não em df_xlsx
    indices_novos = df_csv[~df_csv['Indice'].isin(df_xlsx['Indice'])]

    # Verifica se há realmente novos registros para adicionar
    if not indices_novos.empty:
        # Concatena apenas os registros novos ao df_xlsx
        df_final = pd.concat([df_xlsx, indices_novos], ignore_index=True)
        print(f"\n{len(indices_novos)} novos registros adicionados ao df_xlsx.")
        print("\nPrimeiras linhas do df_final após a adição dos novos registros:")
        # print(df_final.tail()) # Exibe as últimas linhas (os registros adicionados)
    else:
        print("\nNenhum novo registro encontrado no df_csv para adicionar ao df_xlsx.")
        df_final = df_xlsx # Mantém o df_xlsx original
else:
    print("Erro: A coluna 'Indice' não existe em um ou ambos os DataFrames. A comparação e adição não foram realizadas.")
    df_final = df_xlsx # Mantém o df_xlsx original caso a coluna 'Indice' não exista


Nenhum novo registro encontrado no df_csv para adicionar ao df_xlsx.


# Etapa 6

Salvar o df_final em um arquivo XLSX na pasta de output

In [76]:
import os
import shutil
from datetime import datetime

pasta_output = '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input'
pasta_processed = '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Processed'
nome_arquivo_xlsx = os.path.basename(arquivo_xlsx)
arquivo_output_xlsx = os.path.join(pasta_output, nome_arquivo_xlsx)

try:
    # Salva o df_final para um arquivo XLSX na pasta de output
    df_final.to_excel(arquivo_output_xlsx, index=False)
    print(f"\nDataFrame df_final salvo com o mesmo nome do arquivo original em: {arquivo_output_xlsx}")

    # Move os arquivos CSV para a pasta Processed com timestamp no nome
    arquivos_csv = glob.glob(os.path.join(pasta_input, '*.csv'))
    now = datetime.now().strftime("%Y%m%d_%H%M%S") # Obtém a data e hora atual no formato YYYYMMDD_HHMMSS
    for arquivo_csv_para_mover in arquivos_csv:
        try:
            nome_arquivo_csv, extensao = os.path.splitext(os.path.basename(arquivo_csv_para_mover))
            novo_nome_arquivo_csv = f"{nome_arquivo_csv}_{now}{extensao}"
            destino = os.path.join(pasta_processed, novo_nome_arquivo_csv)
            shutil.move(arquivo_csv_para_mover, destino)
            print(f"Arquivo CSV movido para: {destino}")
        except Exception as e:
            print(f"Erro ao mover o arquivo CSV {arquivo_csv_para_mover}: {e}")

except Exception as e:
    print(f"\nOcorreu um erro ao salvar o arquivo XLSX ou ao mover os arquivos CSV: {e}")


DataFrame df_final salvo com o mesmo nome do arquivo original em: /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input/BB_extrato.xlsx
Arquivo CSV movido para: /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Processed/2025-05-extrato_20250510_185009.csv


# Etapa 7

Criar engine de conexão

In [80]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

# Credenciais do banco de dados
dbname = "neondb"
user = "neondb_owner"
password = "npg_ayeAjRZ8sPD7"
host = "ep-rough-shape-a85bihtu-pooler.eastus2.azure.neon.tech"
port = "5432"

# Criar a string de conexão com o PostgreSQL
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Criar a conexão com o banco de dados
engine = create_engine(connection_string)

# Etapa 8

Criar DF com dados da planilha

In [87]:
import pandas as pd
import os

# Caminho da pasta contendo os arquivos
folder_path = '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/02-BB/Input'

# Obter lista de arquivos na pasta com a extensão .xls ou .xlsx
arquivos = [f for f in os.listdir(folder_path) if f.endswith('.xls') or f.endswith('.xlsx')]

# Lista para armazenar os DataFrames processados
df_list = []

# Lista fixa com os nomes das colunas
nomes_colunas = [
    'mesano', 'data_lanc', 'origem', 'descricao', 'data_balancete', 'num_doc', 'valor', 'idcategoria', 'observacao', 'idformapagto', 'dfluxo', 'indice'
]

# Função para aplicar os nomes fixos nas colunas
def aplicar_nomes_colunas(df, novos_nomes):
    if len(novos_nomes) == len(df.columns):
        df.columns = novos_nomes
    else:
        print(f"Erro: A quantidade de novos nomes não corresponde ao número de colunas em {arquivo}.")
    return df

# Iterar sobre os arquivos da pasta
for arquivo in arquivos:
    # Caminho completo do arquivo
    file_path = os.path.join(folder_path, arquivo)

    # Ler o arquivo Excel (independente de ser .xls ou .xlsx)
    df = pd.read_excel(file_path, header=None)  # Não utiliza um cabeçalho por enquanto

    # Encontrar a linha que contém o cabeçalho (a que tem 'data' na primeira coluna)
    header_row = df[df.iloc[:, 1].str.contains('Data', case=False, na=False)].index[0]

    # Ler novamente o arquivo, agora utilizando o cabeçalho correto
    df = pd.read_excel(file_path)
    #df = pd.read_excel(file_path, header=header_row)

    # Aplicar os nomes fixos nas colunas
    df = aplicar_nomes_colunas(df, nomes_colunas)

    # Adicionar uma coluna com o nome do arquivo
    df['nome_arquivo'] = arquivo

    # Adicionar o DataFrame à lista
    df_list.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df_combined = pd.concat(df_list, ignore_index=True)

# Ordenar o DataFrame pela coluna 'data'
df_combined['data_lanc'] = pd.to_datetime(df_combined['data_lanc'], errors='coerce', dayfirst=True)  # Converter a coluna 'data' para datetime
df_combined = df_combined.sort_values(by='data_lanc')  # Ordenar pela coluna 'data'

# Adicionar uma coluna 'data_carga' com a data atual
df_combined['data_carga'] = pd.to_datetime('today').normalize()  # 'normalize' para deixar a hora como 00:00:00

colunas_finais = ['data_lanc', 'descricao', 'num_doc', 'valor', 'idcategoria', 'observacao', 'nome_arquivo', 'data_carga']

df_filtrado = df_combined[colunas_finais]

# Exibir as primeiras linhas do DataFrame combinado com a nova coluna
# df_filtrado.head()

# exibir a quantidade de linhas para cada arquivo
# print(df_filtrado.groupby('nome_arquivo').size())


<ipython-input-87-3aca1ae5685c>:54: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_combined['data_lanc'] = pd.to_datetime(df_combined['data_lanc'], errors='coerce', dayfirst=True)  # Converter a coluna 'data' para datetime


,data_lanc,descricao,num_doc,valor,idcategoria,observacao,nome_arquivo,data_carga
0,2019-02-25,Saldo Anterior,0,140.44,NaN,NaN,BB_extrato.xlsx,2025-05-10
1,2022-03-30,Pix - Enviado,33001,-120.00,Outr03,não identificado,BB_extrato.xlsx,2025-05-10
2,2022-03-30,Saldo Anterior,0,20.44,NaN,NaN,BB_extrato.xlsx,2025-05-10
3,2022-03-31,S A L D O,0,20.44,NaN,NaN,BB_extrato.xlsx,2025-05-10
4,2022-04-07,BB RF LP Automático,1200002,187.20,Rec09,resgate aplicação,BB_extrato.xlsx,2025-05-10


# Etapa 9

Gravar dados no DB

In [89]:
from sqlalchemy import text

# Função para excluir dados antigos e inserir novos, com relatórios
def excluir_e_inserir(df, engine):
    with engine.connect() as conn:
        # Verificar se a tabela existe no banco
        check_table_query = text("""
            SELECT EXISTS (
                SELECT FROM information_schema.tables
                WHERE table_schema = 'fluxo'
                AND table_name = 'bbrasil'
            );
        """)
        table_exists = conn.execute(check_table_query).scalar()

        if table_exists:
            # Se a tabela existir, deletar os dados e gravar novamente
            delete_query = text(f"DELETE FROM fluxo.bbrasil;")
            conn.execute(delete_query)
            conn.commit()
            df.to_sql('bbrasil', engine, schema='fluxo', if_exists='append', index=False)
        else:
            # Se a tabela não existir, considera que nenhum arquivo foi importado ainda
            df.to_sql('bbrasil', engine, schema='fluxo', if_exists='append', index=False)

    # Relatório de execução
    print(f"Processamento concluído.")

# Chamar a função para excluir dados antigos e inserir os novos
excluir_e_inserir(df_filtrado, engine)


Processamento concluído.
